In [1]:
! pip install sentence-transformers

In [2]:
!pip install imbalanced-learn

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

c:\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
pip install kaggle

In [5]:
import os
from zipfile import ZipFile

# Define the path to your kaggle.json file
kaggle_json_path = r"C:\CapStone New\kaggle (1).json"

In [6]:
import shutil

# Ensure the .kaggle directory exists
kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)

# Copy the kaggle.json file to the .kaggle directory
shutil.copy(kaggle_json_path, os.path.join(kaggle_dir, 'kaggle.json'))


print("kaggle.json has been copied to ~/.kaggle/ directory")

kaggle.json has been copied to ~/.kaggle/ directory


In [7]:
# Define the dataset and download path
dataset = 'arshkon/linkedin-job-postings' 
download_path = r'C:\CapStone New\kaggle_datasets'

In [8]:
import kaggle 
# Ensure the download directory exists
os.makedirs(download_path, exist_ok=True)

# Download the dataset
kaggle.api.dataset_download_files(dataset, path=download_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/arshkon/linkedin-job-postings


In [9]:
# List files in the download directory to verify the actual file names
for root, dirs, files in os.walk(download_path):
    for file in files:
        print(os.path.join(root, file))

C:\CapStone New\kaggle_datasets\postings.csv
C:\CapStone New\kaggle_datasets\preprocessed_benefits_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_companies.csv
C:\CapStone New\kaggle_datasets\preprocessed_companies_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_company_industries_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_company_specialities_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_employee_counts_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_industries.csv
C:\CapStone New\kaggle_datasets\preprocessed_industries_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_job_industries_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_job_skills_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_postings.csv
C:\CapStone New\kaggle_datasets\preprocessed_postings_encoded.csv
C:\CapStone New\kaggle_datasets\preprocessed_postings_nlp.csv
C:\CapStone New\kaggle_datasets\preprocessed_salaries.csv
C:\CapStone New\kaggle_d

In [10]:
# Paths to individual CSV files
datasets = {
    'postings': os.path.join(download_path, 'postings.csv'),
    'companies': os.path.join(download_path, 'companies', 'companies.csv'),
    'company_industries': os.path.join(download_path, 'companies', 'company_industries.csv'),
    'company_specialities': os.path.join(download_path, 'companies', 'company_specialities.csv'),
    'employee_counts': os.path.join(download_path, 'companies', 'employee_counts.csv'),
    'benefits': os.path.join(download_path, 'jobs', 'benefits.csv'),
    'job_industries': os.path.join(download_path, 'jobs', 'job_industries.csv'),
    'job_skills': os.path.join(download_path, 'jobs', 'job_skills.csv'),
    'salaries': os.path.join(download_path, 'jobs', 'salaries.csv'),
    'industries': os.path.join(download_path, 'mappings', 'industries.csv'),
    'skills': os.path.join(download_path, 'mappings', 'skills.csv')
}



In [23]:
# Load each CSV file into a pandas DataFrame
dataframes = {name: pd.read_csv(path) for name, path in datasets.items()}

# Function to print head, describe, info, and shape
def print_dataframe_info(df, name):
    print(f"Dataset: {name}")
    print("\nHead:")
    print(df.head(), "\n")
    print("Describe:")
    print(df.describe(), "\n")
    print("Info:")
    df.info()
    print("\nShape:")
    print(df.shape, "\n")
    print("="*80 + "\n")

    

# Print information for each DataFrame
for name, df in dataframes.items():
    print_dataframe_info(df, name)
    input("Press Enter to continue to the next DataFrame...")  # Pauses between each DataFrame


Dataset: postings

Head:
     job_id            company_name  \
0    921716   Corcoran Sawyer Smith   
1   1829192                     NaN   
2  10998357  The National Exemplar    
3  23221523  Abrams Fensterman, LLP   
4  35982263                     NaN   

                                               title  \
0                              Marketing Coordinator   
1                  Mental Health Therapist/Counselor   
2                        Assitant Restaurant Manager   
3  Senior Elder Law / Trusts and Estates Associat...   
4                                 Service Technician   

                                         description  max_salary pay_period  \
0  Job descriptionA leading real estate firm in N...        20.0     HOURLY   
1  At Aspen Therapy and Wellness , we are committ...        50.0     HOURLY   
2  The National Exemplar is accepting application...     65000.0     YEARLY   
3  Senior Associate Attorney - Elder Law / Trusts...    175000.0     YEARLY   
4  Looki

DATA PREPROCESSING

In [12]:
# Function to find and print missing values
def find_missing_values(dataframes):
    for name, df in dataframes.items():
        print(f"Missing values in {name}:")
        print(df.isnull().sum())
        print("="*80 + "\n")

# Find and print missing values
find_missing_values(dataframes)

Missing values in postings:
job_id                             0
company_name                    1719
title                              0
description                        7
max_salary                     94056
pay_period                     87776
location                           0
company_id                      1717
views                           1689
med_salary                    117569
min_salary                     94056
formatted_work_type                0
applies                       100529
original_listed_time               0
remote_allowed                108603
job_posting_url                    0
application_url                36665
application_type                   0
expiry                             0
closed_time                   122776
formatted_experience_level     29409
skills_desc                   121410
listed_time                        0
posting_domain                 39968
sponsored                          0
work_type                          0
currency  

In [13]:
# Function to remove duplicates
def remove_duplicates(dataframes):
    for name, df in dataframes.items():
        print(f"Removing duplicates for {name}")
        dataframes[name] = df.drop_duplicates()

# Remove duplicates
remove_duplicates(dataframes)

Removing duplicates for postings
Removing duplicates for companies
Removing duplicates for company_industries
Removing duplicates for company_specialities
Removing duplicates for employee_counts
Removing duplicates for benefits
Removing duplicates for job_industries
Removing duplicates for job_skills
Removing duplicates for salaries
Removing duplicates for industries
Removing duplicates for skills


In [14]:

# Handle missing values in the companies dataset
companies = dataframes['companies'].copy()
companies = companies.dropna(subset=['name'])
companies['description'] = companies['description'].fillna('No description available')
companies['company_size'] = companies['company_size'].fillna(companies['company_size'].median())
companies['state'] = companies['state'].fillna(companies['state'].mode()[0])
companies = companies.dropna(subset=['city'])
companies['zip_code'] = companies['zip_code'].fillna('00000')
companies['address'] = companies['address'].fillna('No address available')

# Handle missing values in the salaries dataset
salaries = dataframes['salaries'].copy()
salaries['max_salary'] = salaries['max_salary'].fillna(salaries['max_salary'].median())
salaries['med_salary'] = salaries['med_salary'].fillna(salaries['med_salary'].median())
salaries['min_salary'] = salaries['min_salary'].fillna(salaries['min_salary'].median())

# Handle missing values in the postings dataset
postings = dataframes['postings'].copy()
postings = postings.dropna(subset=['company_name'])
postings['description'] = postings['description'].fillna('No description available')
postings['max_salary'] = postings['max_salary'].fillna(postings['max_salary'].median())
postings['med_salary'] = postings['med_salary'].fillna(postings['med_salary'].median())
postings['min_salary'] = postings['min_salary'].fillna(postings['min_salary'].median())
postings['pay_period'] = postings['pay_period'].fillna(postings['pay_period'].mode()[0])
postings = postings.dropna(subset=['company_id'])
postings['views'] = postings['views'].fillna(postings['views'].median())
postings['applies'] = postings['applies'].fillna(postings['applies'].median())
postings['remote_allowed'] = postings['remote_allowed'].fillna(postings['remote_allowed'].mode()[0])
postings['application_url'] = postings['application_url'].fillna('No application URL')
postings['formatted_experience_level'] = postings['formatted_experience_level'].fillna(postings['formatted_experience_level'].mode()[0])
postings['skills_desc'] = postings['skills_desc'].fillna('No skills listed')
postings['posting_domain'] = postings['posting_domain'].fillna('No domain')
postings['currency'] = postings['currency'].fillna(postings['currency'].mode()[0])
postings['compensation_type'] = postings['compensation_type'].fillna(postings['compensation_type'].mode()[0])

# Handle missing values in the industries dataset
industries = dataframes['industries'].copy()
industries['industry_name'] = industries['industry_name'].fillna('Unknown Industry')

# Save the preprocessed dataframes if needed
companies.to_csv(os.path.join(download_path, 'preprocessed_companies.csv'), index=False)
salaries.to_csv(os.path.join(download_path, 'preprocessed_salaries.csv'), index=False)
postings.to_csv(os.path.join(download_path, 'preprocessed_postings.csv'), index=False)
industries.to_csv(os.path.join(download_path, 'preprocessed_industries.csv'), index=False)

# Update the dataframes dictionary with the preprocessed data
dataframes['companies'] = companies
dataframes['salaries'] = salaries
dataframes['postings'] = postings
dataframes['industries'] = industries

print("Missing values handled and dataframes saved.")

Missing values handled and dataframes saved.


In [15]:
# Print head of each preprocessed dataframe
for name, df in dataframes.items():
    print(f"Head of {name} dataset:")
    print(df.head(), "\n")

Head of postings dataset:
      job_id               company_name  \
0     921716      Corcoran Sawyer Smith   
2   10998357     The National Exemplar    
3   23221523     Abrams Fensterman, LLP   
5   91700727  Downtown Raleigh Alliance   
6  103254301                 Raw Cereal   

                                               title  \
0                              Marketing Coordinator   
2                        Assitant Restaurant Manager   
3  Senior Elder Law / Trusts and Estates Associat...   
5           Economic Development and Planning Intern   
6                                           Producer   

                                         description  max_salary pay_period  \
0  Job descriptionA leading real estate firm in N...        20.0     HOURLY   
2  The National Exemplar is accepting application...     65000.0     YEARLY   
3  Senior Associate Attorney - Elder Law / Trusts...    175000.0     YEARLY   
5  Job summary:The Economic Development & Plannin...        20

NLP PREPROCESSING

In [16]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):
        return text
    # Lowercasing
    text = text.lower()
    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenizing
    words = word_tokenize(text)
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Lemmatizing
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # Join words back into a single string
    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Apply the preprocessing function to the description columns
postings['description'] = postings['description'].apply(preprocess_text)
postings['skills_desc'] = postings['skills_desc'].apply(preprocess_text)
companies['description'] = companies['description'].apply(preprocess_text)

# Display the first few rows to verify the preprocessing
print(postings[['description', 'skills_desc']].head())
print(companies['description'].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaisy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaisy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaisy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                         description  \
0  job descriptiona leading real estate firm new ...   
2  national exemplar accepting application assist...   
3  senior associate attorney elder law trust esta...   
5  job summarythe economic development planning i...   
6  company descriptionraw cereal creative design ...   

                                         skills_desc  
0  requirement seeking college graduate student a...  
2  currently accepting resume foh asisstant resta...  
3  position requires baseline understanding onlin...  
5                                       skill listed  
6                                       skill listed  
0    ibm work create create technologist developer ...
1    every day million people feel impact intellige...
2    official linkedin hewlett packard enterprise g...
3    ’ cloud technology company provides organizati...
4    accenture leading global professional service ...
Name: description, dtype: object


Merging Datasets

In [34]:
# Merge company-related datasets
company_data = dataframes['companies'].merge(dataframes['company_industries'], on='company_id', how='left')
company_data = company_data.merge(dataframes['company_specialities'], on='company_id', how='left')
company_data = company_data.merge(dataframes['employee_counts'], on='company_id', how='left')

In [35]:
# Merge job-related datasets
job_data = dataframes['postings'].merge(dataframes['job_industries'], on='job_id', how='left')
job_data = job_data.merge(dataframes['job_skills'], on='job_id', how='left')
job_data = job_data.merge(dataframes['salaries'], on='job_id', how='left')
job_data = job_data.merge(dataframes['benefits'], on='job_id', how='left')

In [36]:
# Combine postings with company and job data
final_data = job_data.merge(company_data, on='company_id', how='left')

In [37]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Function to encode categorical variables
def encode_categorical_variables(df):
    # Select categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    # Apply One-Hot Encoding to columns with low cardinality
    low_cardinality_cols = [col for col in categorical_cols if df[col].nunique() < 100]
    high_cardinality_cols = [col for col in categorical_cols if df[col].nunique() >= 100]
    
    df_low_card = pd.get_dummies(df[low_cardinality_cols], drop_first=True)
    
    # Apply Label Encoding to columns with high cardinality
    label_encoders = {}
    for col in high_cardinality_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
    
    # Drop the original low cardinality columns and concatenate the encoded columns
    df = df.drop(columns=low_cardinality_cols)
    df = pd.concat([df, df_low_card], axis=1)
    
    return df, label_encoders

In [38]:
# Apply encoding to the postings DataFrame
postings_encoded, postings_label_encoders = encode_categorical_variables(postings)

# Apply encoding to the company DataFrame
company_data_encoded, company_label_encoders = encode_categorical_variables(companies)



# Verify encoding
print(postings_encoded.head())
print(company_data_encoded.head())


      job_id  company_name  title  description  max_salary  location  \
0     921716          5381  35491        47135        20.0      6228   
2   10998357         21335   4945        65212     65000.0      1390   
3   23221523           557  57093        89627    175000.0      5332   
5   91700727          6440  19531        51524        20.0      6288   
6  103254301         17409  45415        16132    300000.0      7678   

   company_id  views  med_salary  min_salary  ...  work_type_FULL_TIME  \
0   2774458.0   20.0       25.37        17.0  ...                 True   
2  64896719.0    8.0       25.37     45000.0  ...                 True   
3    766262.0   16.0       25.37    140000.0  ...                 True   
5   1481176.0    9.0       25.37        14.0  ...                False   
6  81942316.0    7.0       25.37     60000.0  ...                False   

   work_type_INTERNSHIP  work_type_OTHER  work_type_PART_TIME  \
0                 False            False                F

In [39]:
import os

# Define the path where you want to save the cleaned dataset
output_path = os.path.join(download_path, 'cleaned_final_data.csv')

# Save the final dataset to a CSV file
final_data.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to {output_path}")

Cleaned dataset saved to C:\CapStone New\kaggle_datasets\cleaned_final_data.csv
